This notebook will develope tools to calculate the pressure from the hydrostatic relation. 

In [2]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

from nowcast import analyze
from salishsea_tools import viz_tools, psu_tools

import scipy.interpolate as scipy_int

import grid_time
import gsw

%matplotlib inline

Plan: 
* Use hydrostatic approximation to calculate pressure:
$ \frac{\partial p}{\partial z} = -\rho g $ 
with $p(z=0) = 0$
* For now, calculate density with the function Susan gave me. How should this change with TEOS-10?
* To enforce $p(z=0)=0$, I think it is best to interpolate rho onto w grid and then integrate for pressure with upper BC and then interpolate back to t grid 

# Loading

In [3]:
mesh_mask = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/grid/mesh_mask_SalishSea2.nc')
grid = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')

bathy = grid.variables['Bathymetry'][:]
# Sample file for testing calculations
fT = nc.Dataset('/results/SalishSea/nowcast/01apr15/SalishSea_1h_20150401_20150401_grid_T.nc')

In [4]:
tmask = mesh_mask.variables['tmask'][:]
e3t_0 = mesh_mask.variables['e3t'][:]
gdept_0 = mesh_mask.variables['gdept'][:]
gdepw_0 = mesh_mask.variables['gdepw'][:]

In [5]:
ssh = fT.variables['sossheig'][:]
sal = fT.variables['vosaline'][:]
temp = fT.variables['votemper'][:]
rho = psu_tools.calculate_density(temp, sal)

In [6]:
e3t_t, e3w_t, gdept_t, gdepw_t = grid_time.calculate_vertical_grids(e3t_0[0,...], tmask[0,...], ssh)

In [7]:
def interp_depth_time_dependent(var, depth, depth_int):
   
    var_int = np.zeros(var.shape)
    for t in np.arange(var_int.shape[0]):
        for j in np.arange(var_int.shape[2]):
            for i in np.arange(var_int.shape[3]):
                var_int[t,:,j,i] = np.interp(depth_int[t,:,j,i], depth[t,:,j,i], var[t,:,j,i],right = np.nan)
    return var_int

In [8]:
def calculate_pressure(rho, e3t_t, gdept_t, tmask):
    """Integrate density to calulate pressure.
    Use time-dependent scale factors
    
    returns pressure in Pascals"""
    g = 9.81 # is there a constant module that I an grab this from?
    
    # integrate density to get pressure
    p = np.cumsum(g*rho*e3t_t*tmask, axis=1)
    p = p - np.expand_dims(g*rho[:,0,...]*gdept_t[:,0,...], axis=1) # Attempt to approximate p(z=0)=0.
    
    return p
    

In [9]:
t=2
p_t = calculate_pressure(rho[0:t], e3t_t[0:t], gdept_t[0:t], tmask)

In [10]:
p_t[0,:,400,300]

array([   5081.97537416,   15264.71065816,   25504.05833731,
         35771.60396324,   46046.96165661,   56320.31685063,
         66586.84866391,   76844.68407262,   87094.80132038,
         97340.56650632,  107589.17846981,  117855.13098334,
        128167.18247194,  138581.89624688,  149209.30717496,
        160262.41149617,  172151.62408805,  185664.02926869,
        202286.49356069,  224749.78986749,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387])

In [11]:
lat = fT.variables['nav_lon'][:]
p_gsw = gsw.p_from_z(-gdept_t[0:t], lat)
p_gsw[0,:,400,300]

MemoryError: 

In [ ]:
diff = (p_t*10**(-4) - p_gsw)*tmask

In [ ]:
diff[0,:,400,300]